<a href="https://colab.research.google.com/github/vydra-v-getrax/ML_2023/blob/main/Regression_Texts_Konovalova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

На этом занятии мы попробуем задачу регрессии. Данные в этой же папке, будем тренироваться на датасете фильмов с IMDB

Перед обучением обучением модели, нужно подготовить данные:

- найти\собрать данные
- почистить и предобработать
- преобразовать в матрицы 


In [ ]:
# импорты необходимых библиотек
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

# import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [ ]:
data = pd.read_csv('IMDB-Movie-Data.csv')
print(data.shape)

data.head(3)

(1000, 12)


Rank                    Title                     Genre  \
0     1  Guardians of the Galaxy   Action,Adventure,Sci-Fi   
1     2               Prometheus  Adventure,Mystery,Sci-Fi   
2     3                    Split           Horror,Thriller   

                                         Description            Director  \
0  A group of intergalactic criminals are forced ...          James Gunn   
1  Following clues to the origin of mankind, a te...        Ridley Scott   
2  Three girls are kidnapped by a man with a diag...  M. Night Shyamalan   

                                              Actors  Year  Runtime (Minutes)  \
0  Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...  2014                121   
1  Noomi Rapace, Logan Marshall-Green, Michael Fa...  2012                124   
2  James McAvoy, Anya Taylor-Joy, Haley Lu Richar...  2016                117   

   Rating   Votes  Revenue (Millions)  Metascore  
0     8.1  757074              333.13       76.0  
1     7.0  485820              126.46       65.0  
2     7.3  157606              138.12       62.0

## Что делать с NaN?
Есть 3 варианта

In [ ]:
# 1. Убрать строки с NaN
print(data.isna().any())
data.shape

Rank                  False
Title                 False
Genre                 False
Description           False
Director              False
Actors                False
Year                  False
Runtime (Minutes)     False
Rating                False
Votes                 False
Revenue (Millions)     True
Metascore              True
dtype: bool


(1000, 12)

In [ ]:
print(data.shape)
tmp = data.dropna()
tmp.shape

(1000, 12)


(838, 12)

In [ ]:
# 2. Превратить NaN в 0
print(data.shape)
tmp = data.fillna(0)
print(tmp.shape)

(1000, 12)
(1000, 12)


In [ ]:
# 3. Превратить NaN в средние значения по колонке

# вычисляем средние для колонок с пустыми значениями
meta_mean = data.Metascore.mean()
rev_mean = data['Revenue (Millions)'].mean()

#заменяем пустоты на средние значения
data.Metascore.fillna(meta_mean, inplace=True)
data['Revenue (Millions)'].fillna(rev_mean, inplace=True)

# проверяем присутствие NaN
data.isna().any()

Rank                  False
Title                 False
Genre                 False
Description           False
Director              False
Actors                False
Year                  False
Runtime (Minutes)     False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
dtype: bool

## Подготовка данных

Попробуем предсказывать рейтинг фильма по данным его описания, года, длины в минутах и кассовых сборов

Колонка "Rating" станет **целевой переменной, или таргетом** (y)<br>
Остальных данные будут **обучающей выборкой** (X)

In [ ]:
data.Description

0      A group of intergalactic criminals are forced ...
1      Following clues to the origin of mankind, a te...
2      Three girls are kidnapped by a man with a diag...
3      In a city of humanoid animals, a hustling thea...
4      A secret government agency recruits some of th...
                             ...                        
995    A tight-knit team of rising investigators, alo...
996    Three American college students studying abroa...
997    Romantic sparks occur between two dance studen...
998    A pair of friends embark on a mission to reuni...
999    A stuffy businessman finds himself trapped ins...
Name: Description, Length: 1000, dtype: object

In [ ]:
# подготовим описания фильмов
data["text"] = data.Description.apply(lambda x: x.lower().split()) 

data["text"]

0      [a, group, of, intergalactic, criminals, are, ...
1      [following, clues, to, the, origin, of, mankin...
2      [three, girls, are, kidnapped, by, a, man, wit...
3      [in, a, city, of, humanoid, animals,, a, hustl...
4      [a, secret, government, agency, recruits, some...
                             ...                        
995    [a, tight-knit, team, of, rising, investigator...
996    [three, american, college, students, studying,...
997    [romantic, sparks, occur, between, two, dance,...
998    [a, pair, of, friends, embark, on, a, mission,...
999    [a, stuffy, businessman, finds, himself, trapp...
Name: text, Length: 1000, dtype: object

In [ ]:
data.text.values

array([list(['a', 'group', 'of', 'intergalactic', 'criminals', 'are', 'forced', 'to', 'work', 'together', 'to', 'stop', 'a', 'fanatical', 'warrior', 'from', 'taking', 'control', 'of', 'the', 'universe.']),
       list(['following', 'clues', 'to', 'the', 'origin', 'of', 'mankind,', 'a', 'team', 'finds', 'a', 'structure', 'on', 'a', 'distant', 'moon,', 'but', 'they', 'soon', 'realize', 'they', 'are', 'not', 'alone.']),
       list(['three', 'girls', 'are', 'kidnapped', 'by', 'a', 'man', 'with', 'a', 'diagnosed', '23', 'distinct', 'personalities.', 'they', 'must', 'try', 'to', 'escape', 'before', 'the', 'apparent', 'emergence', 'of', 'a', 'frightful', 'new', '24th.']),
       list(['in', 'a', 'city', 'of', 'humanoid', 'animals,', 'a', 'hustling', 'theater', "impresario's", 'attempt', 'to', 'save', 'his', 'theater', 'with', 'a', 'singing', 'competition', 'becomes', 'grander', 'than', 'he', 'anticipates', 'even', 'as', 'its', "finalists'", 'find', 'that', 'their', 'lives', 'will', 'never', 

In [ ]:
input_text = list(data.text.values)

In [ ]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]
documents[10:12]

[TaggedDocument(words=['the', 'adventures', 'of', 'writer', 'newt', 'scamander', 'in', 'new', "york's", 'secret', 'community', 'of', 'witches', 'and', 'wizards', 'seventy', 'years', 'before', 'harry', 'potter', 'reads', 'his', 'book', 'in', 'school.'], tags=[10]),
 TaggedDocument(words=['the', 'story', 'of', 'a', 'team', 'of', 'female', 'african-american', 'mathematicians', 'who', 'served', 'a', 'vital', 'role', 'in', 'nasa', 'during', 'the', 'early', 'years', 'of', 'the', 'u.s.', 'space', 'program.'], tags=[11])]

обучаем модель на текстах описаний фильмов (можно поизменять параметры)

In [ ]:
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [ ]:
model.save("D2V.model") # сохранение модели

In [ ]:
# так можно посмотреть на векторы текстов, на которых училась модель
# индекс [] около documents -- это индекс текста из датасета

model.dv[documents[0].tags[0]]


Теперь нужно добавить векторы в датасет с остальными параметрами

In [ ]:
# создадим список с векторами для каждого текста
vectors = []
for x in documents:
    vec = list(model[x.tags][0])
    vectors.append(vec)

In [ ]:
# так получим датафрейм, где все компоненты векторов в отдельных столбцах
split_df = pd.DataFrame(vectors,
                        columns=['v1', 'v2', 'v3','v4',"v5"])

split_df


v1        v2        v3        v4        v5
0    1.664000  3.103669  2.429229 -4.210670 -0.782168
1    1.300252  3.070507  2.333138 -4.279568 -0.671380
2    1.806727  3.135074  2.189884 -4.559013 -0.590260
3    1.439255  3.000284  2.221531 -4.524175 -0.702553
4    1.919748  3.509732  2.663648 -4.965741 -0.377711
..        ...       ...       ...       ...       ...
995  0.117624  0.202358  0.041334 -0.008874 -0.036378
996  0.081031  0.276267 -0.051899 -0.375093  0.129633
997 -0.030110  0.364167  0.244481 -0.449511  0.143345
998  0.241176  0.090991 -0.007657 -0.507057  0.049657
999  0.194874  0.080019  0.116005 -0.353762 -0.114460

[1000 rows x 5 columns]

In [ ]:
# теперь добавим его к основному датафрейму
result = data.join(split_df, how='left')
result.shape

(1000, 18)

In [ ]:
result

Rank                    Title                     Genre  \
0       1  Guardians of the Galaxy   Action,Adventure,Sci-Fi   
1       2               Prometheus  Adventure,Mystery,Sci-Fi   
2       3                    Split           Horror,Thriller   
3       4                     Sing   Animation,Comedy,Family   
4       5            Suicide Squad  Action,Adventure,Fantasy   
..    ...                      ...                       ...   
995   996     Secret in Their Eyes       Crime,Drama,Mystery   
996   997          Hostel: Part II                    Horror   
997   998   Step Up 2: The Streets       Drama,Music,Romance   
998   999             Search Party          Adventure,Comedy   
999  1000               Nine Lives     Comedy,Family,Fantasy   

                                           Description              Director  \
0    A group of intergalactic criminals are forced ...            James Gunn   
1    Following clues to the origin of mankind, a te...          Ridley Scott   
2    Three girls are kidnapped by a man with a diag...    M. Night Shyamalan   
3    In a city of humanoid animals, a hustling thea...  Christophe Lourdelet   
4    A secret government agency recruits some of th...            David Ayer   
..                                                 ...                   ...   
995  A tight-knit team of rising investigators, alo...             Billy Ray   
996  Three American college students studying abroa...              Eli Roth   
997  Romantic sparks occur between two dance studen...            Jon M. Chu   
998  A pair of friends embark on a mission to reuni...        Scot Armstrong   
999  A stuffy businessman finds himself trapped ins...      Barry Sonnenfeld   

                                                Actors  Year  \
0    Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...  2014   
1    Noomi Rapace, Logan Marshall-Green, Michael Fa...  2012   
2    James McAvoy, Anya Taylor-Joy, Haley Lu Richar...  2016   
3    Matthew McConaughey,Reese Witherspoon, Seth Ma...  2016   
4    Will Smith, Jared Leto, Margot Robbie, Viola D...  2016   
..                                                 ...   ...   
995  Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...  2015   
996  Lauren German, Heather Matarazzo, Bijou Philli...  2007   
997  Robert Hoffman, Briana Evigan, Cassie Ventura,...  2008   
998  Adam Pally, T.J. Miller, Thomas Middleditch,Sh...  2014   
999  Kevin Spacey, Jennifer Garner, Robbie Amell,Ch...  2016   

     Runtime (Minutes)  Rating   Votes  Revenue (Millions)  Metascore  \
0                  121     8.1  757074          333.130000       76.0   
1                  124     7.0  485820          126.460000       65.0   
2                  117     7.3  157606          138.120000       62.0   
3                  108     7.2   60545          270.320000       59.0   
4                  123     6.2  393727          325.020000       40.0   
..                 ...     ...     ...                 ...        ...   
995                111     6.2   27585           82.956376       45.0   
996                 94     5.5   73152           17.540000       46.0   
997                 98     6.2   70699           58.010000       50.0   
998                 93     5.6    4881           82.956376       22.0   
999                 87     5.3   12435           19.640000       11.0   

                                                  text        v1        v2  \
0    [a, group, of, intergalactic, criminals, are, ...  1.664000  3.103669   
1    [following, clues, to, the, origin, of, mankin...  1.300252  3.070507   
2    [three, girls, are, kidnapped, by, a, man, wit...  1.806727  3.135074   
3    [in, a, city, of, humanoid, animals,, a, hustl...  1.439255  3.000284   
4    [a, secret, government, agency, recruits, some...  1.919748  3.509732   
..                                                 ...       ...       ...   
995  [a, tight-knit, team, of, rising, investigator...  0.117624  0.202358   
996  

In [ ]:
# переопределим датасет, оставив только важное

data_sm = result[['Runtime (Minutes)',"Year",
                'Rating', 'Votes',
                'Revenue (Millions)','Metascore',"v1","v2","v3","v4","v5"]
              ]


data_sm.head(3)

Runtime (Minutes)  Year  Rating   Votes  Revenue (Millions)  Metascore  \
0                121  2014     8.1  757074              333.13       76.0   
1                124  2012     7.0  485820              126.46       65.0   
2                117  2016     7.3  157606              138.12       62.0   

         v1        v2        v3        v4        v5  
0  1.664000  3.103669  2.429229 -4.210670 -0.782168  
1  1.300252  3.070507  2.333138 -4.279568 -0.671380  
2  1.806727  3.135074  2.189884 -4.559013 -0.590260

## Подготавливаем матрицы

In [ ]:
# определяем X и y

X = data_sm.drop(["Rating"],axis=1).values 

display(X, X.shape)

array([[ 1.21000000e+02,  2.01400000e+03,  7.57074000e+05, ...,
         2.42922902e+00, -4.21066999e+00, -7.82167912e-01],
       [ 1.24000000e+02,  2.01200000e+03,  4.85820000e+05, ...,
         2.33313775e+00, -4.27956820e+00, -6.71380103e-01],
       [ 1.17000000e+02,  2.01600000e+03,  1.57606000e+05, ...,
         2.18988395e+00, -4.55901337e+00, -5.90259731e-01],
       ...,
       [ 9.80000000e+01,  2.00800000e+03,  7.06990000e+04, ...,
         2.44481146e-01, -4.49511170e-01,  1.43344611e-01],
       [ 9.30000000e+01,  2.01400000e+03,  4.88100000e+03, ...,
        -7.65713537e-03, -5.07057309e-01,  4.96573150e-02],
       [ 8.70000000e+01,  2.01600000e+03,  1.24350000e+04, ...,
         1.16004527e-01, -3.53762120e-01, -1.14459969e-01]])

(1000, 10)

In [ ]:
data_sm.isna().any()

Runtime (Minutes)     False
Year                  False
Rating                False
Votes                 False
Revenue (Millions)    False
Metascore             False
v1                    False
v2                    False
v3                    False
v4                    False
v5                    False
dtype: bool

In [ ]:
y = data_sm['Rating'].values # отдельно вынесли массив со значениями скорости ветра
y.shape

(1000,)

Иногда бывает полезно [нормализовать](https://en.wikipedia.org/wiki/Normalization_(statistics)) данные: это позволяет исправить ситуацию, когда признаки представлены в разных единацах измерения. 
Для этого используется StandardScaler. 

До нормализации:

In [ ]:
list(X[0])

[121.0,
 2014.0,
 757074.0,
 333.13,
 76.0,
 1.6640000343322754,
 3.1036691665649414,
 2.4292290210723877,
 -4.210669994354248,
 -0.782167911529541]

In [ ]:
# использзуем стандартизатор
sc = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(sc.fit_transform(X), y, random_state=42)

После:

In [ ]:
list(sc.fit_transform(X)[0])

[0.4163497512303056,
 0.37979525138136244,
 3.1126899627963738,
 2.5961363010556906,
 1.0233613578368184,
 5.348091129939745,
 5.396043249935101,
 5.648766702345885,
 -5.065361131303684,
 -5.167030145715259]

теперь с данными удобнее работать и обучать

In [ ]:
# задаем модель регрессора
# силу регуляризации можно варьировать параметром alpha
regressor = Ridge() 


# обучаем
regressor.fit(X_train, y_train)

Ridge()

In [ ]:
# давайте предскажем результат для тестовой выборки

y_preds = regressor.predict(X_test)

### оценка результатов алгоритма

В качестве метрики будем использовать [среднюю абсолютную ошибку](https://www.youtube.com/watch?v=ZejnwbcU8nw). Она показывает отклонение от правильного ответа в тех же единах измерения

*(а вообще есть [разные способы](https://towardsdatascience.com/what-are-the-best-metrics-to-evaluate-your-regression-model-418ca481755b))*

In [ ]:
mean_absolute_error(y_test, y_preds) 

0.48313174756403765

In [ ]:
mean_squared_error(y_test, y_preds)

0.46020370286458395

In [ ]:
mean_squared_error(y_test, y_preds, squared=False)

0.6783831534351247

Попробуйте разные значения для параметра регуляризации alpha при обучении модели. Как они влияют на величину ошибки?

# Отсюда новое (копия колаба)

In [ ]:
mae_baseline, mse_baseline, rmse_baseline = 0.48313174756403765, 0.46020370286458395, 0.6783831534351247

print("На семинаре были получены такие значения ошибок: ")
print("mae: {}, \nmse: {},\nrmse:{}".format(mae_baseline, mse_baseline, rmse_baseline))

На семинаре были получены такие значения ошибок: 
mae: 0.48313174756403765, 
mse: 0.46020370286458395,
rmse:0.6783831534351247


In [ ]:
# импорты необходимых библиотек
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt
# %matplotlib inline

# import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error


In [ ]:
# Попробуем использовать другую предобработку текста

import nltk
# nltk.download('stopwords')
# nltk.download('punkt')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import string
import re

PUNCT = string.punctuation
STOP_WORDS = set(stopwords.words('english'))
porter = PorterStemmer()


def preprocess(text) -> str:
    word_tokens = word_tokenize(text)
    filtered_sentence = [porter.stem(w).lower() for w in word_tokens]
    filered_sentence = [w for w in filtered_sentence if not w.lower() in STOP_WORDS]
    filtered_sentence = [w for w in filtered_sentence if w not in PUNCT]
    
#     result = []
#     for w in filtered_sentence:
#         if not re.search('[^0-9]', w):
#             result.append('DIGIT')
#         else:
#             result.append(w)
    return filtered_sentence

## Apply new preprocessing & tokenization

In [ ]:
data = pd.read_csv('IMDB-Movie-Data.csv')
print(data.shape)

data.head(3)


(1000, 12)


Rank                    Title                     Genre  \
0     1  Guardians of the Galaxy   Action,Adventure,Sci-Fi   
1     2               Prometheus  Adventure,Mystery,Sci-Fi   
2     3                    Split           Horror,Thriller   

                                         Description            Director  \
0  A group of intergalactic criminals are forced ...          James Gunn   
1  Following clues to the origin of mankind, a te...        Ridley Scott   
2  Three girls are kidnapped by a man with a diag...  M. Night Shyamalan   

                                              Actors  Year  Runtime (Minutes)  \
0  Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...  2014                121   
1  Noomi Rapace, Logan Marshall-Green, Michael Fa...  2012                124   
2  James McAvoy, Anya Taylor-Joy, Haley Lu Richar...  2016                117   

   Rating   Votes  Revenue (Millions)  Metascore  
0     8.1  757074              333.13       76.0  
1     7.0  485820              126.46       65.0  
2     7.3  157606              138.12       62.0

In [ ]:
def vectorize(vector_size=2, window=3, min_count=1, prep=True) -> pd.DataFrame():
    if prep:
        data['preprocessed'] = data.Description.apply(lambda x: preprocess(x))
    else:
        data['preprocessed'] = data.Description.apply(lambda x: x.lower().split())
    input_text = list(data.preprocessed.values)
    documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(input_text)]
    model = Doc2Vec(documents, vector_size=vector_size, window=window, min_count=min_count, workers=1)
    model.save("D2V.model") # сохранение модели
    
        # создадим список с векторами для каждого текста
    vectors = []
    for x in documents:
        vec = list(model[x.tags][0])
        vectors.append(vec)

    # Добавляем в датафрейм
    split_df = pd.DataFrame(vectors, 
                            columns=[f'vec{i}' for i in range(len(vectors[0]))])
    
    # Объединим с предыдущим датафреймом

    cols_to_keep = ['Runtime (Minutes)',
                    "Year",
                    'Rating', 
                    'Votes',
                    'Revenue (Millions)',
                    'Metascore']
    cols_to_keep.extend(list(split_df.columns))
    result = data.join(split_df, how='left')[cols_to_keep]
    y = result.Rating
    X = result.drop(columns=["Rating"])
    X['Revenue (Millions)'].fillna(X['Revenue (Millions)'].mean(), inplace=True)
    X['Metascore'].fillna(X['Metascore'].mean(), inplace=True)
    
    # нормализация

    sc = StandardScaler()

    X_train, X_test, y_train, y_test = train_test_split(sc.fit_transform(X), y, random_state=42)
    
    return X_train, X_test, y_train, y_test


### Найдем лучшую модель и подберем гиперпараметры путем перебора

Я сравнила два способа: с препроцессингом (токенизация, стемминг, удаление стоп-слов, удаление пунктуации) и без него (с базовым сплитом по пробелам). Возможно, препроцессинг был выбран неверно, но качество оказалось хуже после подбора гиперпараметров с препроцессингом, поэтому в итоговых результатах я вернулась просто к делению по пробелам.

In [ ]:

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

In [ ]:
X_train, X_test, y_train, y_test = vectorize(vector_size=5, window=1, min_count=1, prep=False)

In [ ]:
FOLDS = 3

# lr = LinearRegression()
ridge = Ridge()
lasso = Lasso()

kfold = KFold(n_splits=FOLDS)
bests = pd.DataFrame(columns=['alpha'])

for name, model in {"ridge": ridge, "lasso": lasso}.items():
    grid = GridSearchCV(model, 
                        cv=kfold,
                        param_grid=[{'alpha': [0.01, 0.015, 0.02, 0.05, 10, 23, 30, 3, 0.1, 0.5, 0.8]}])
    grid.fit(X_train, y_train)
    bests.loc[name, 'alpha'] = grid.best_params_['alpha']


In [ ]:
bests

alpha
ridge    23
lasso  0.01

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

for name in ['ridge', 'lasso', 'lr']:
    model_base = False
    if name == "ridge":
        model = Ridge(alpha=bests.loc[name, 'alpha'])
        model_base = Ridge()
    elif name == "lasso":
        model = Lasso(alpha=bests.loc[name, 'alpha'])
        model_base = Lasso()
    else:
        model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_pred, y_test)
    mse = mean_squared_error(y_pred, y_test)
    rmse = mean_squared_error(y_pred, y_test, squared=False)

    bests.loc[name, 'mae'] = round(mae, 4)
    bests.loc[name, 'mse'] = round(mse, 4)
    bests.loc[name, 'rmse'] = round(rmse, 4)
    
    if model_base:
        model_base.fit(X_train, y_train)
        y_pred = model_base.predict(X_test)

        mae = mean_absolute_error(y_pred, y_test)
        mse = mean_squared_error(y_pred, y_test)
        rmse = mean_squared_error(y_pred, y_test, squared=False)

        bests.loc[name, 'mae_no_alpha'] = round(mae, 4)
        bests.loc[name, 'mse_no_alpha'] = round(mse, 4)
        bests.loc[name, 'rmse_no_alpha'] = round(rmse, 4)

### Результаты

Путем перебора я сравнила два способа: с препроцессингом (токенизация, стемминг, удаление стоп-слов, удаление пунктуации) и без него (с базовым сплитом по пробелам). Возможно, препроцессинг был выбран неверно, но качество оказалось хуже после подбора гиперпараметров с препроцессингом, поэтому в итоговых результатах я вернулась просто к делению по пробелам. 

In [ ]:
bests

alpha     mae     mse    rmse  mae_no_alpha  mse_no_alpha  rmse_no_alpha
ridge    23  0.4832  0.4618  0.6795        0.4829        0.4598         0.6781
lasso  0.01  0.4833  0.4638  0.6810        0.7736        0.9734         0.9866
lr      NaN  0.4829  0.4597  0.6780           NaN           NaN            NaN

In [ ]:
# Сравним с исходными (Ridge без настройки гиперпараметров и препроцессинга):
print("mae: {0:10.4f}".format(mae_baseline))
print("mse: {0:10.4f}".format(mse_baseline))
print("rmse: {0:10.4f}".format(rmse_baseline))

mae:     0.4831
mse:     0.4602
rmse:     0.6784


Линейная регрессия без настроек показала лучшие результаты, чем бейзлайн (Ridge). Качество зависело от модели и от настроек векторизации (vector_size=5, window=1). Однако другие модели оказались ниже бейзлайна. Я пробовала менять разные настройки векторизатора (размер вектора документа, размер окна, минимальный порог для токена), но качество становилось хуже. В таблице выше приведен лучший результат. 

На данных с препроцессингом: 

In [ ]:
X_train, X_test, y_train, y_test = vectorize(vector_size=5, window=1, min_count=1, prep=True)

In [ ]:
FOLDS = 3

# lr = LinearRegression()
ridge = Ridge()
lasso = Lasso()

kfold = KFold(n_splits=FOLDS)
bests = pd.DataFrame(columns=['alpha'])

for name, model in {"ridge": ridge, "lasso": lasso}.items():
    grid = GridSearchCV(model, 
                        cv=kfold,
                        param_grid=[{'alpha': [0.01, 0.015, 0.02, 0.05, 10, 23, 30, 3, 0.1, 0.5, 0.8]}])
    grid.fit(X_train, y_train)
    bests.loc[name, 'alpha'] = grid.best_params_['alpha']


In [ ]:
bests

alpha
ridge    23
lasso  0.01

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

for name in ['ridge', 'lasso', 'lr']:
    model_base = False
    if name == "ridge":
        model = Ridge(alpha=bests.loc[name, 'alpha'])
        model_base = Ridge()
    elif name == "lasso":
        model = Lasso(alpha=bests.loc[name, 'alpha'])
        model_base = Lasso()
    else:
        model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_pred, y_test)
    mse = mean_squared_error(y_pred, y_test)
    rmse = mean_squared_error(y_pred, y_test, squared=False)

    bests.loc[name, 'mae'] = round(mae, 4)
    bests.loc[name, 'mse'] = round(mse, 4)
    bests.loc[name, 'rmse'] = round(rmse, 4)
    
    if model_base:
        model_base.fit(X_train, y_train)
        y_pred = model_base.predict(X_test)

        mae = mean_absolute_error(y_pred, y_test)
        mse = mean_squared_error(y_pred, y_test)
        rmse = mean_squared_error(y_pred, y_test, squared=False)

        bests.loc[name, 'mae_no_alpha'] = round(mae, 4)
        bests.loc[name, 'mse_no_alpha'] = round(mse, 4)
        bests.loc[name, 'rmse_no_alpha'] = round(rmse, 4)

In [ ]:
bests

alpha     mae     mse    rmse  mae_no_alpha  mse_no_alpha  rmse_no_alpha
ridge    23  0.4831  0.4647  0.6817        0.4824        0.4627         0.6802
lasso  0.01  0.4846  0.4654  0.6822        0.7736        0.9734         0.9866
lr      NaN  0.4824  0.4626  0.6801           NaN           NaN            NaN

Видно, что с препроцессингом качетсво стало выше на модели Ridge по метрике mean absolute error, но остальные показатели стали ниже. Вероятно, для наших данных нужно использовать другой способ препроцессинга (не удалять стоп-слова, заменить стемминг на лемматизацию и др.). Я попробовала несколько разных вариантов, привожу оптимальный.

В целом можно сказать, что подбор параметра alpha не значительно улучшил качество работы моделей. Возможные причины:
* датасет довольно маленький, модель быстро переобучается
* нужна более тщательная подготовка данных (например, составить список стоп-слов, специфичных для датасета, подобрать оптимальную очистку от посторонних символов, избавиться от именованных сущностей в тексте)
* можно использовать дополнительные признаки: имена режиссеров и лидирующих актеров как категориальные переменные, добавить названия фильмов как текстовые признаки. 